<a href="https://colab.research.google.com/github/kang1006/theme-premium/blob/master/Theme-Premium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# 변수 저장하기
tickers = ['QYLD', 'BIL', 'AOR']
start_date = '2000-01-01'  # 시작일
end_date = '2023-06-30'  # 종료일

obs_period = np.array([3, 6, 9, 12])
skip_period = np.array([1])

In [22]:
# 데이터 불러오기
data = yf.download(tickers, start=start_date, end=end_date)['Close']
data.dropna(inplace=True)
print(data)

[*********************100%***********************]  3 of 3 completed
                  AOR        BIL       QYLD
Date                                       
2013-12-12  37.669998  91.580002  25.040001
2013-12-13  37.669998  91.580002  25.020000
2013-12-16  37.840000  91.559998  25.049999
2013-12-17  37.799999  91.559998  25.070000
2013-12-18  38.150002  91.580002  25.030001
...               ...        ...        ...
2023-06-23  50.830002  91.720001  17.480000
2023-06-26  50.840000  91.730003  17.350000
2023-06-27  50.930000  91.750000  17.559999
2023-06-28  51.080002  91.760002  17.580000
2023-06-29  51.090000  91.790001  17.570000

[2402 rows x 3 columns]


In [24]:
# 모멘텀 계산하기
new_data = data.copy()
for obs in obs_period:
    obs_month = obs * 22  # 월 단위를 일 단위로 변환
    for skip in skip_period:
        skip_month = skip * 22  # 월 단위를 일 단위로 변환

        roc = (new_data[tickers].pct_change(periods=obs_month) - new_data[tickers].pct_change(periods=skip_month)).rename(columns=lambda x: f"ROC_{x}_{obs}m_{skip}m")
        new_data = new_data.join(roc)

        abs_momentum = np.where((new_data[f"ROC_QYLD_{obs}m_{skip}m"].notna()) & (new_data[f"ROC_BIL_{obs}m_{skip}m"].notna()), (new_data[f"ROC_QYLD_{obs}m_{skip}m"] > new_data[f"ROC_BIL_{obs}m_{skip}m"]).astype(int), np.nan)
        abs_momentum = pd.DataFrame(abs_momentum, index=new_data.index, columns=[f"abs_momentum_{obs}m_{skip}m"])
        new_data = new_data.join(abs_momentum)

In [27]:
# 투자비중 만들기
weight = pd.DataFrame(index=new_data.index, columns=['QYLD', 'BIL'])
abs_momentum_columns = [col for col in new_data.columns if "abs_momentum" in col]
weight['QYLD'] = new_data[abs_momentum_columns].mean(axis=1)
weight['BIL'] = 1 - weight['QYLD']

# # (6)
# portfolio = pd.DataFrame(index=data.index, columns=['Portfolio'])
# portfolio['Portfolio'] = weight['QYLD'] * data['QYLD'] + weight['BIL'] * data['BIL']

# # AOR US 데이터 불러오기
# aor_data = yf.download('AOR', start=data.index[0], end=data.index[-1])['Close']

# # 리밸런싱 주기 설정
# rebalancing_frequency = 0.1  # 10% 변할 때마다 리밸런싱

# # 포트폴리오 성과 계산
# portfolio['AOR'] = aor_data
# portfolio['Returns'] = portfolio['Portfolio'].pct_change()
# portfolio['Cumulative Returns'] = (1 + portfolio['Returns']).cumprod()

# # 리밸런싱 시점 확인
# rebalancing_dates = portfolio[portfolio['Returns'].abs() >= rebalancing_frequency].index

# # 그래프 그리기
# plt.figure(figsize=(12, 6))
# plt.plot(portfolio['Cumulative Returns'], label='Portfolio')
# plt.plot(portfolio['AOR'] / portfolio['AOR'].iloc[0], label='AOR US')
# plt.legend(loc='upper left')
# plt.xlabel('Date')
# plt.ylabel('Cumulative Returns')
# plt.title('Portfolio Performance')
# plt.show()

# # 포트폴리오 성과 메트릭 계산
# portfolio_metrics = portfolio.loc[rebalancing_dates, ['Cumulative Returns', 'Returns']]
# portfolio_metrics['Rebalancing Dates'] = rebalancing_dates

# # 결과 출력
# print(portfolio_metrics)

,QYLD,BIL
Date,,
2013-12-12,NaN,NaN
2013-12-13,NaN,NaN
2013-12-16,NaN,NaN
2013-12-17,NaN,NaN
2013-12-18,NaN,NaN
...,...,...
2023-06-23,0.75,0.25
2023-06-26,0.50,0.50
2023-06-27,0.50,0.50


Date
2013-12-12   NaN
2013-12-13   NaN
2013-12-16   NaN
2013-12-17   NaN
2013-12-18   NaN
              ..
2023-06-23   NaN
2023-06-26   NaN
2023-06-27   NaN
2023-06-28   NaN
2023-06-29   NaN
Name: abs_momentum_3m_1m, Length: 2402, dtype: float64